# Идея

Для человека не составит труда определить по спутниковому снимку плотность застройки, примерную численость населения в данном квадрате, или, например, торговый центр в котором в выходной день всегда много народа. Все эти признаки прямо или косвенно влияют на количество экстренных вызовов из данной области.

Картинка для привлечения внимания! Снизу представлены две области, на первой из них сетка дала достаточно низкий результат по звонкам а на второй очень высокий =) 

<img src="least_call.png" width="300">
<img src="most_call.png" width="300"> 

# 1) Загрузка текстовых данных <a name='1'></a>

In [480]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

import pandas 

df_zones = pandas.read_json('whole_MOSCOW.json')


In [482]:
df_zones.head()

,calls_wd0,calls_wd1,calls_wd2,calls_wd3,calls_wd4,calls_wd5,calls_wd6,lat_c,lon_c
0,0.003006,0.000000e+00,6.885802e-04,0.003006,0.000000e+00,0.001374,0.007052,55.380482,37.003077
1,0.000765,0.000000e+00,1.176913e-03,0.000765,0.000000e+00,0.001374,0.002818,55.380514,37.010966
2,0.001322,0.000000e+00,1.020885e-03,0.001322,0.000000e+00,0.001374,0.003871,55.380545,37.018855
3,0.000815,0.000000e+00,7.802124e-04,0.000815,0.000000e+00,0.001374,0.002912,55.380576,37.026744
4,0.000783,1.822000e-07,7.287000e-07,0.000782,8.602000e-07,0.001374,0.002851,55.380606,37.034633


# 2) Загрузка "спутниковых" снимков <a name='2'></a>

Итоговые картинки имеют размер 512х512 и физические размеры квадрата со стороной около 500 метров

In [483]:
from PIL import Image
import urllib
import os

def load_img(x,y,i):

    width, height = 512,512
    #Create a new image of the size require
    map_img = Image.new('RGB', (width,height))
    
    url = 'https://maps.googleapis.com/maps/api/staticmap?maptype=satellite&center={},{}&zoom=16&scale=2&size=512x512&key=your_api_key'.format(str(x),str(y))
    
    current_tile = str(x)+'-'+str(y)
    urllib.request.urlretrieve(url, current_tile)
    
    im = Image.open(current_tile)
    map_img.paste(im)
    os.remove(current_tile)
    map_img.save("photos/{}.png".format(i))
    

В данном примере я обкачиваю только квадраты, которые имеют target == 1. Можно также попробовать обкачать все данные и использовать их в обучении.

In [484]:
df_zones_for_nn = df_zones

In [486]:
import tqdm

for i in tqdm.tqdm(df_zones_for_nn.iterrows()):
    load_img(i[1][-2],i[1][-1],str(i[0]))
    

# 3) Тренировка нейросети <a name='3'></a>

В качестве таргета может выступать как среднее количество звонков по всем дням недели, так и отдельно по рабочим и выходным дням(два выхода или даже две отдельные модели). Также можно сделать 7 выходов по всем дням недели. В данном примере будет рассмотрены только среднее количество звонков по всем дням.

In [20]:
X = df_zones.query('is_test == 0 & is_target == 1').index.values
print(X.shape)
y =  df_zones.query('is_test == 0 & is_target == 1')[['calls_daily']].values

(4374,)


In [46]:
# преобразование таргет переменной
y = np.log1p(y)

In [48]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(3936,)
(3936, 1)
(438,)
(438, 1)


In [487]:
from PIL import Image
from keras.utils import Sequence


def load_img_as_array(img_path):
    
    img = Image.open('photos/'+img_path+'.png')
    
    return np.asarray(img)


class DataGen(Sequence):
    def __init__(self, xx, yy, batch_size):
        self.xx = xx
        self.yy = yy
        self.batch_size = batch_size

    
    def __len__(self):
        return len(self.xx) // self.batch_size
    
    def __getitem__(self, i):
        X_list = []
        for j in range(self.batch_size):
            
                x = load_img_as_array(str(self.xx[i*self.batch_size + j]))
                X_list.append(x)
        
        X_batch = np.array(X_list)
        y_batch = self.yy[i*self.batch_size:(i+1)*self.batch_size]
        
        return X_batch, y_batch

Using TensorFlow backend.


In [488]:
from keras.models import Model,Input
from keras.layers import Dense, Dropout, Reshape, Concatenate
from keras.applications.resnet50 import ResNet50

def create_model(crop_size):
    
    base_model = ResNet50(include_top=False, weights='imagenet', input_shape=(crop_size, crop_size, 3),pooling='max')    
    
    output_1 = base_model.output
    
     
    x = Dense(2048, activation='relu', name='fc1')(output_1)
    x = Dropout(0.3, name='dropout_fc1')(x)
    x = Dense(512, activation='relu', name='fc2')(x)
    x = Dropout(0.3, name='dropout_fc2')(x)
    x = Dense(64, activation='relu', name='fc3')(x)
    x = Dropout(0.3, name='dropout_fc3')(x)
    predictions = Dense(1, activation="linear", name='predictions')(x)

    model = Model(inputs=base_model.input, outputs=predictions)    
    #model.summary()
    
    return model

In [489]:
CROP_SIZE = 512

model = create_model(CROP_SIZE)

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau,TensorBoard
from keras.optimizers import Adam

model_folder = './models_experiment'
model_name = 'calls_daily_nn_resnet50'
batch_size = 8

opt = Adam(lr=1e-4)
model.compile(optimizer=opt, loss='mse', metrics=['mse'])

monitor = 'val_mean_squared_error'
model_path = os.path.join(model_folder, model_name + '-epoch-{epoch:03d}' + '-val_mse-{val_mean_squared_error:.3f}' + '.hdf5')

reduce_lr = ReduceLROnPlateau(monitor=monitor, factor=0.5, patience=1, min_lr=1e-9, epsilon = 0.00001, verbose=1)
model_checkpoint = ModelCheckpoint(model_path, monitor=monitor, save_best_only=True, verbose=0)
early_stopping = EarlyStopping(monitor='val_mean_squared_error', patience=5, verbose=0)


callbacks = [reduce_lr, early_stopping, model_checkpoint]

history = model.fit_generator(
    generator=DataGen(X_train, y_train, batch_size),
    steps_per_epoch=len(X_train)//batch_size,
    epochs=1000,
    
    validation_data=DataGen(X_test, y_test, batch_size),
    validation_steps=len(X_test)//batch_size,

    callbacks=callbacks,
    max_queue_size = 10,
    workers=4)

# 4) Предсказания модели <a name='4'></a>

Валидация

In [ ]:
from keras.models import load_model
import glob

model_folder = './models_experiment'

models = glob.glob(os.path.join(model_folder, 'calls_daily_nn_resnet50*.hdf5'))

print(sorted(models)[-1])
loaded_model = load_model(sorted(models)[-1])
print("Loaded model from disk")

In [ ]:
pred_val = loaded_model.predict_generator(DataGen(X_test, y_test, df_features.loc[X_test, :], 1))

In [42]:
from scipy.stats import kendalltau

valid_score = kendalltau(y_test, pred_val).correlation
print('Validation score:', valid_score)

Validation score: 0.6586098635104689


Предсказания для теста

In [38]:
idx_test = df_zones.query('is_test == 1').index

pred_test = loaded_model_bow.predict_generator(DataGen(idx_test, idx_test, df_features.loc[idx_test, :], 1),verbose = 1)

8473/8473 [==============================] - 386s 46ms/step


In [39]:
pred_test = pred_test.reshape(-1)

In [ ]:
target_columns = ['calls_wd{}'.format(d) for d in range(7)]
df_test_predictions = pandas.DataFrame(collections.OrderedDict([
    (column_name, pred_test)
    for column_name in target_columns
]), index=idx_test)

df_test_predictions.to_csv('sample_submission.csv')
df_test_predictions.head(5)

# 5) Заключение <a name='5'></a>

Был рассмотрен один из возможных подходов использования спутниковых снимков для предсказания количества экстренных вызовов из заданного квадрата. Можно поэкспериментировать и попробовать следующее:

- Другая архитектруа нейросети
- Добавить еще других текстовых признаков
- Другая функция потерь (MAE или даже попробовать ранжирование)
- Докинуть различных аугментаций (сейчас только горизонтальные и вертикальные флипы)
- Другое приобразование целевой переменной (Бокса-Кокса)
